In [1]:
import os
import sys

sys.path.append(os.path.abspath('..'))

from early_exit.deploy_early_exit import *
from early_exit.train_early_exit import *

import torch
import torch.nn as nn
from cifar10_models.vgg import vgg11_bn, vgg13_bn, vgg16_bn, vgg19_bn
from shared.dataloaders import *
from shared.models import *
from copy import deepcopy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Untrained model

# Pretrained model
pretrained_model = vgg19_bn(pretrained=True)
pretrained_model.eval() # for evaluation

Files already downloaded and verified
Files already downloaded and verified


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [2]:
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)
class SplitModel(nn.Module):
    def __init__(self,pretrained_model):
        super(SplitModel, self).__init__()  # Call the
        self.device_side_net = deepcopy(pretrained_model.features[:6])
        self.server_side_net = deepcopy(pretrained_model.features[6:14])
        
        self.cloud_net = nn.Sequential(
            deepcopy(pretrained_model.features[14:]),
            deepcopy(pretrained_model.avgpool),
            Flatten(),
            deepcopy(pretrained_model.classifier)
        )
        self.net = nn.Sequential(
            self.device_side_net,
            self.server_side_net,
            self.cloud_net
        )
        
    def forward(self, x):
        x = self.net(x)
        return x

In [3]:
model = SplitModel(pretrained_model)
input_shape = (1,3, 32, 32)
eenet = EarlyExitNet(model.net,input_shape,device,[0.9,0.9,0.9])
test_all_exits_accuracy(eenet,testloader)